In [41]:
import numpy as np
import random

## The softmax function

The softmax function allows us to transform a vector of real numbers into a probability distribution. Its formula is:

$$
\mathrm{softmax}(x, i) = \frac{e^x_i}{\sum_{j=1}^{n}{e^x_j}}
$$

$e^x$ gets huge quickly, and makes us run into numerical limitation. We can show that 
$$
\mathrm{softmax}(x,i) = \frac{e^{x_i-c}}{\sum_{j=1}^{n}{e^{x_j - c}}}
$$
which allows us to subtract the maximum value of our vector from each element to avoid numerical issues.

In [61]:
def softmax(x):
    orig_shape = x.shape
    
    if len(x.shape) > 1:
        # Matrix
        c = -np.array([np.max(x, axis=1)]).T
        e_x = np.exp(x + c)
        _sum = e_x.sum(axis=1)
        x = e_x / _sum[:, None]
    else:
        # Vector
        c = -np.max(x)
        e_x = np.exp(x + c)
        _sum = np.sum(e_x)
        x = e_x / _sum
        
    assert x.shape == orig_shape
    return x

When implementing derivative functions, it's quite useful to check intermediate results before going wildly off tangent. An easy way to do this is to compute the numerical gradient, and compare the two values.

We use a very simple numerical calculation of the gradient:

$$
\mathrm{grad}(f(x)) = \frac{f(x+h) - f(x-h)}{2h}
$$

We use [numpy multi-index iteration](https://docs.scipy.org/doc/numpy-1.14.0/reference/arrays.nditer.html#tracking-an-index-or-multi-index).

## Naive gradient checking

In [149]:
def check_gradient(f, x):
    rnd_state = random.getstate()
    random.setstate(rnd_state)

    fx, grad = f(x)
    h = 1e-4
    
    # x can be a vector or a matrix, so we want to compute the derivative according to each element.
    # This is straight out of the cs224n code.
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        
        x[ix] += h
        random.setstate(rnd_state)

        fx_1, _ = f(x)
        random.setstate(rnd_state)

        x[ix] -= 2 * h
        fx_2, _ = f(x)
        x[ix] += h
        
        numgrad = (fx_1 - fx_2) / (2 * h)
        
        # we can now check the numerical gradient against the computed gradient.
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print("Gradient check failed at index {}: {} should be {}".format(ix, grad[ix], numgrad))
            return
        
        it.iternext()
    
    print("Gradients seem to be ok!")

Let's check out the gradient checking.

In [150]:
def test_gradient_f(x):
    return 2. * x[0], np.ones_like(x) * 2.

check_gradient(test_gradient_f, np.array([1.]))

Gradients seem to be ok!


## Row normalization

It is useful to normalize the rows of our word2vec embeddings on each training iteration.

In [151]:
def normalize_rows(x):
    return x / np.sqrt(np.sum(x * x, axis=1))[:, None]

## SGD

Here we implement stochastic gradient descent. We iterate over our data in batches of pairs (input, target). We compute our own result (forward-pass), take the gradient wrt the target, and propagate the gradient backwards, updating our variables.

In [152]:
def sgd(f, x0, step, iterations, postprocessing=None, PRINT_EVERY=100):
    start_iter = 0
    
    ANNEAL_EVERY = 20000
    
    if postprocessing is None:
        postprocessing = lambda x: x
        
    x = x0
    
    for i in range(start_iter + 1, iterations + 1):
        cost, gradient = f(x)
        x -= step * gradient
        postprocessing(x)
        
        if i % PRINT_EVERY == 0:
            print("iteration {}: {:.4f}".format(i, cost))

        if i % ANNEAL_EVERY == 0:
            step *= 0.5
            
    return x

Let's test the gradient descent on a very simple function:

$$
\sum_{i=1}^{n}{x_i}^2
$$

In [153]:
def quad(x):
    return np.sum(x ** 2), x * 2

In [154]:
sgd(f=quad, x0=0.5, step=0.01, iterations=1000, PRINT_EVERY=100)

iteration 100: 0.0046
iteration 200: 0.0001
iteration 300: 0.0000
iteration 400: 0.0000
iteration 500: 0.0000
iteration 600: 0.0000
iteration 700: 0.0000
iteration 800: 0.0000
iteration 900: 0.0000
iteration 1000: 0.0000


8.414836786079764e-10

In [155]:
sgd(f=quad, x0=0.0, step=0.01, iterations=1000, PRINT_EVERY=100)

iteration 100: 0.0000
iteration 200: 0.0000
iteration 300: 0.0000
iteration 400: 0.0000
iteration 500: 0.0000
iteration 600: 0.0000
iteration 700: 0.0000
iteration 800: 0.0000
iteration 900: 0.0000
iteration 1000: 0.0000


0.0

In [156]:
sgd(f=quad, x0=-1.5, step=0.001, iterations=1000, PRINT_EVERY=100)

iteration 100: 1.5137
iteration 200: 1.0142
iteration 300: 0.6796
iteration 400: 0.4554
iteration 500: 0.3051
iteration 600: 0.2044
iteration 700: 0.1370
iteration 800: 0.0918
iteration 900: 0.0615
iteration 1000: 0.0412


-0.2025967836700259

## Skip gram

We implement the skip-gram model. In skip-gram, we are going to predict the context words for every center word.
Skip-gram uses softmax with log-entropy as a loss function. We derive and implement the necessary functions (derivative according to the predicted vector, and derivative according to all the output vectors).

In [157]:
def softmax_cost_and_gradient(predicted, target, output_vectors):
    y_hat = softmax(np.dot(predicted, output_vectors.T))
    cost = -np.log(y_hat[target])
    
    grad = np.outer(y_hat, predicted)
    grad[target] -= predicted
    grad_predicted = -output_vectors[target] + np.sum(y_hat[:, None] * output_vectors, axis=0)
    
    return cost, grad_predicted, grad

Let's test the softmax_cost

In [158]:
Ninner = 3
Nwords = 5
vc = np.random.rand(Ninner)
target = 1
output_vectors = np.random.rand(Ninner * Nwords).reshape((Nwords, Ninner))

def softmax_cost_and_gradient_predicted(vc, output_vectors):
    cost, grad_predicted, grad = softmax_cost_and_gradient(vc, target, output_vectors)
    return cost, grad_predicted

def softmax_cost_and_gradient_grad(vc, output_vectors):
    cost, grad_predicted, grad = softmax_cost_and_gradient(vc, target, output_vectors)
    return cost, grad

check_gradient(lambda output_vectors: softmax_cost_and_gradient_grad(vc, output_vectors), output_vectors)
check_gradient(lambda vc: softmax_cost_and_gradient_predicted(vc, output_vectors), vc)

Gradients seem to be ok!
Gradients seem to be ok!


In [165]:
def skip_gram(current_word, context_size, context_words, tokens, input_vectors, output_vectors):
    cost = 0.
    grad_in = np.zeros(input_vectors.shape)
    grad_out = np.zeros(output_vectors.shape)
    
    center_index = tokens[current_word]
    
#     print("current_word: {}, context_words: {}".format(current_word, context_words))
    
    for word in context_words:
        target = tokens[word]
        cost_, grad_predicted_, grad_ = softmax_cost_and_gradient(predicted=input_vectors[center_index],
                                                                 target=target,
                                                                 output_vectors=output_vectors)
        cost += cost_
        grad_in[center_index] += grad_predicted_
        grad_out += grad_
        
    return cost, grad_in, grad_out

We can now plug this into our sgd.

In [169]:
def word2vec_sgd(tokens, word_vectors, context_size, dataset):
    batch_size = 50
    cost = 0.0
    grad = np.zeros(word_vectors.shape)

    N = word_vectors.shape[0]
    
    # our word vectors are going to be split in 2 for the purpose of training
    input_vectors = word_vectors[:int(N / 2), :]
    output_vectors = word_vectors[int(N / 2):, :]
    
    for i in range(batch_size):
        context_size_ = random.randint(1, context_size)
        center_word, context_words = dataset.getRandomContext(context_size_)
        
        cost_, grad_in_, grad_out_ = skip_gram(center_word, 
                                               context_size=context_size, 
                                               context_words=context_words,
                                               tokens=tokens, 
                                               input_vectors=input_vectors, 
                                               output_vectors=output_vectors)
        
        cost += cost_ / batch_size
        grad[:int(N / 2), :] += grad_in_ / batch_size
        grad[int(N / 2):, :] += grad_out_ / batch_size
    
#     print("cost: {}, grad: {}".format(cost, grad))
    return cost, grad

Let's do some tests.

In [170]:
dataset = type('dummy', (), {})()

def dummySampleTokenIdx():
    return random.randint(0, 4)

def getRandomContext(C):
    tokens = ["a", "b", "c", "d", "e"]
    return tokens[random.randint(0, 4)], \
           [tokens[random.randint(0, 4)] for i in range(2 * C)]

dataset.sampleTokenIdx = dummySampleTokenIdx
dataset.getRandomContext = getRandomContext

random.seed(31415)
np.random.seed(9265)
dummy_vectors = normalize_rows(np.random.randn(10, 3))
dummy_tokens = dict([("a", 0), ("b", 1), ("c", 2), ("d", 3), ("e", 4)])

In [171]:
check_gradient(lambda vec: word2vec_sgd(dummy_tokens, vec, context_size=5, dataset=dataset), dummy_vectors)

Gradients seem to be ok!


## Input dataset

We can now take a look at the input data we are going to be using.    

In [173]:
from utils.treebank import StanfordSentiment

In [174]:
dataset = StanfordSentiment()
tokens = dataset.tokens()
n_words = len(tokens)

The dataset is a big dict of words to their index.

In [175]:
type(tokens)

dict

In [176]:
list(tokens.keys())[:10]

[b'six-packs',
 b'banking',
 b'contributed',
 b'sex-as-war',
 b'anakin',
 b'career-defining',
 b'tension',
 b'muccino',
 b'sequence',
 b'nightmares']

In [177]:
tokens[b'banking']

7233

In [179]:
dataset = StanfordSentiment()
tokens = dataset.tokens()
n_words = len(tokens)

dim_vectors = 10
context_size = 5

random.seed(31415)
np.random.seed(9265)

word_vectors = np.concatenate(
    ((np.random.rand(n_words, dim_vectors) - 0.5) / dim_vectors, np.zeros((n_words, dim_vectors))), 
    axis=0)
word_vectors = sgd(
    f=lambda word_vectors: word2vec_sgd(tokens=tokens, 
                                        word_vectors=word_vectors, 
                                        context_size=context_size, 
                                        dataset=dataset),
    x0=word_vectors,
    step=0.3,
    iterations=40000,
    PRINT_EVERY=10)


iteration 10: 27.0717
iteration 20: 28.0597
iteration 30: 26.4789
iteration 40: 27.2693
iteration 50: 31.0237
iteration 60: 29.2453
iteration 70: 26.4788


KeyboardInterrupt: 